***Deep Autoencoder***

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense
from livelossplot import PlotLossesKeras

**Import dataset** (MNIST)

In [ ]:
(X_train, _), (X_test, _) = mnist.load_data()

max_value = float(X_train.max())
X_train = X_train.astype("float32")/max_value
X_test = X_test.astype("float32")/max_value

# Reshape 28x28 images into vectors of length 784
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

print(X_train.shape)
print(X_test.shape)

In [ ]:
input_dim = 28*28 # 784
encoding_dim = 6*6

**Build model**

In [ ]:
autoencoder = Sequential()

# Encoder layers
autoencoder.add(Dense(4*encoding_dim, input_shape = (input_dim,), activation = "relu"))
autoencoder.add(Dense(2*encoding_dim, activation = "relu"))
autoencoder.add(Dense(1*encoding_dim, activation = "relu"))

# Decoder layers
autoencoder.add(Dense(2*encoding_dim, activation = "relu"))
autoencoder.add(Dense(4*encoding_dim, activation = "relu"))
autoencoder.add(Dense(input_dim, activation = "sigmoid"))

autoencoder.summary()

We can extract the **encoder** model from the autoencoder. The encoder model consists of the first 3 layers in the autoencoder, so let’s extract them to visualize the encoded images.

In [ ]:
input_img = Input(shape = (input_dim,))

encoder_layer1 = autoencoder.layers[0]
encoder_layer2 = autoencoder.layers[1]
encoder_layer3 = autoencoder.layers[2]

encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))

encoder.summary()

**Train model**

In [ ]:
autoencoder.compile(
    optimizer = "adam",
    loss = "binary_crossentropy")

start_time = time.time()

epochs = 10

autoencoder.fit(
    X_train, X_train,
    epochs = epochs,
    batch_size = 256,
    validation_data = (X_test, X_test),
    callbacks = [PlotLossesKeras()])

end_time = time.time() - start_time
print(f"Training time: {end_time} seconds for {epochs} epochs")
print(f"Training time: {end_time/epochs} per epoch on average")

**Save model**

In [ ]:
autoencoder.save("Models/deep_autoencoder.model")
encoder.save("Models/deep_autoencoder_encoder.model")

**Load model**

In [ ]:
autoencoder = load_model("Models/deep_autoencoder.model")
encoder = load_model("Models/deep_autoencoder_encoder.model")

**Display output**

In [ ]:
n = 10
np.random.seed(12)

encoded_imgs = encoder.predict(X_test)
decoded_imgs = autoencoder.predict(X_test)

plt.figure(figsize = (18, 4))

for i in range(n):
    j = np.random.randint(0, len(X_test))
    # original image
    ax = plt.subplot(3, n, i+1)
    ax.set_title(f"[{j}]")
    plt.imshow(X_test[j].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # encoded image
    ax = plt.subplot(3, n, i+1+n)
    plt.imshow(encoded_imgs[j].reshape(6, 6))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # reconstructed image
    ax = plt.subplot(3, n, i+1+2*n)
    plt.imshow(decoded_imgs[j].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
n = 8
seed = lambda k: int(99/(2*k+1)*13) # arbitrary mapping from k to some seed

# k rows of images
for k in range(4):
    np.random.seed(seed(k))
    plt.figure(figsize = (18, 4))
    for i in range(n):
        j = np.random.randint(0, len(X_test))
        ax = plt.subplot(1, n, i+1)
        plt.imshow(encoded_imgs[j].reshape(6, 6))
        ax.set_title(f"[{j}]")
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()